# Data Merging Notebook

#### This notebook merges data from the 20_intermediate_files directory and saves a merged dataset for final analysis. We use a notebook for this instead of a script for interpretability across teammates and easier error checking.

#### Imports

In [1]:
import pandas as pd
import numpy as np


#### Import datasets

In [2]:
# set directory
working_dir = "../20_intermediate_files/"

# set file paths
census = working_dir + "census_df.csv"
fips = working_dir + "fips_df.csv"
vital = working_dir + "vital_clean.csv"
wapo = working_dir + "wapo_clean.csv"


In [3]:
# read in data
census_df = pd.read_csv(census)
fips_df = pd.read_csv(fips, dtype={"FIPS": str, "STATE_FIPS": str})
vital_df = pd.read_csv(vital, dtype={"FIPS": str})
wapo_df = pd.read_csv(wapo, dtype={"YEAR": str, "MONTH": str})


#### Merge Prep

***Census Data is low drama***

In [4]:
# take a look at the data starting with census

# rename county_name to match other data
census_df = census_df.rename(columns={"COUNTY_NAME": "county_name"})

# Rename Miami-Dade County to Dade County
census_df.loc[
    census_df["county_name"] == "MIAMI-DADE COUNTY, FL", "county_name"
] = "DADE COUNTY, FL"


In [5]:
# show counties that contain the word DADE
census_df[census_df["county_name"].str.contains("DADE")]


,county_name,POPULATION,STATE
182,"DADE COUNTY, FL",2496435,FL
247,"DADE COUNTY, GA",16633,GA


In [6]:
print(f"There are {census_df.shape[0]} counties in the census data.")


There are 1039 counties in the census data.


In [64]:
census_df["STATE"].unique()

array(['AL', 'AZ', 'CA', 'FL', 'GA', 'ID', 'MS', 'NM', 'NY', 'OK', 'OR',
       'SC', 'TX', 'WA'], dtype=object)

In [7]:
# census_df.sample(5)


***Fix the FIPS data next to remove the state-only lines***

In [8]:
# add a new column of the int of the FIPS code
fips_df["FIPS_INT"] = fips_df["FIPS"].astype(int)

# drop the row if the FIPS code is divisible by 100
fips_df = fips_df[fips_df["FIPS_INT"] % 100 != 0]


In [9]:
# show fips for counties that contain the word DADE
# fips_df[fips_df["COUNTY_NAME"].str.contains("DADE")]


In [10]:
# FIPS codes
# fips_df.dtypes
# fips_df.sample(5)


In [11]:
print(f"There are {fips_df.shape[0]} counties in the FIPS data set.")


There are 1039 counties in the FIPS data set.


***Clean the Vital DF to add the new merge key with FIPS code***

In [12]:
# show miami dade county
# vital_df[vital_df["FIPS"] == "12086"]
# vital_df[vital_df["FIPS"] == "12025"]

# because it changed

# search the vital_df for FIPS code 12086 and change it to 12025 and rename the county to DADE COUNTY, FL for consistency
vital2 = vital_df.replace("012086", "012025")
vital3 = vital2.replace("MIAMI-DADE COUNTY, FL", "DADE COUNTY, FL")


In [13]:
# show FIPS codes for counties that contain the word DADE
# vital3[vital3["COUNTY_NAME"].str.contains("DADE")]


***FINALLY***

In [14]:
vital_df = vital3.copy()

# if the FIPS code is six characters, remove the leading zero
vital_df["FIPS"] = vital_df["FIPS"].apply(lambda x: x[1:] if len(x) == 6 else x)

# Add a column for the merge key: County_name+year
vital_df["merge_key"] = vital_df["FIPS"] + vital_df["YEAR"].astype(str)


In [15]:
# find fips of 12025 should be many and 12086 - should be null
# vital_df[vital_df["FIPS"] == "12025"]
# vital_df[vital_df["FIPS"] == "12086"]


In [16]:
# check for duplicate merge keys
vital_df["merge_key"].duplicated().sum()


0

In [17]:
print(
    f"There are {vital_df.shape[0]} rows in the vital_df after miami-dade corrections."
)


There are 3064 rows in the vital_df after miami-dade corrections.


No data for DOÑA ANA COUNTY, NM and DEBACA COUNTY, NM at all

In [78]:
"DOÑA ANA COUNTY, NM" in vital_df["COUNTY_NAME"]

False

In [81]:
"DONA ANA COUNTY, NM" in vital_df["COUNTY_NAME"]

False

In [82]:
"DEBACA COUNTY, NM" in vital_df["COUNTY_NAME"]

False

In [18]:
# show vital df dade where fips is either 12025 or 12086
# vital_df[(vital_df["FIPS"] == "12025") | (vital_df["FIPS"] == "12086")]


***Prep the WAPO dataset by adding FIPS to county names and generating merge key***

In [19]:
# groupby the COUNTY_NAME, STATE, and YEAR and sum the DRUG_QUANTITY
wapo_df = (
    wapo_df.groupby(["COUNTY_NAME", "STATE", "YEAR"])
    .agg({"QUANTITY": "sum"})
    .reset_index()
)


In [20]:
# wapo_df.sample(10)


In [21]:
# Add FIPS codes to the wapo_df data by merging on county_name
wapo_df_fips = pd.merge(wapo_df, fips_df, on="COUNTY_NAME", how="left").copy()

# Drop STATE_FIPS and STATE_y
wapo_df_fips = wapo_df_fips.drop(["STATE_FIPS", "STATE_y", "FIPS_INT"], axis=1)

# strip the last character from the FIPS column
wapo_df_fips["FIPS"] = wapo_df_fips["FIPS"].str[:-1]

# Add the same merge key
wapo_df_fips["merge_key"] = wapo_df_fips["FIPS"] + wapo_df_fips["YEAR"].astype(str)

# rename the QUANTITY column to DRUG_QUANTITY
wapo_df_fips = wapo_df_fips.rename(
    columns={"QUANTITY": "DRUG_QUANTITY", "STATE_x": "STATE"}
)

# add a flag column to indicate the source of the data
wapo_df_fips["source"] = 1


In [22]:
# WAPO Data
# wapo_df_fips.sample(5)


In [23]:
# make a list of the unique counties in the wapo_df_fips where FIPS is NaN
missing_counties = wapo_df_fips[wapo_df_fips["FIPS"].isna()]["COUNTY_NAME"].unique()


In [24]:
missing_counties


array(['DE BACA COUNTY, NM', 'DE KALB COUNTY, AL', 'DE SOTO COUNTY, FL',
       'DE WITT COUNTY, TX', 'MIAMI-DADE COUNTY, FL',
       'SAINT CLAIR COUNTY, AL', 'SAINT JOHNS COUNTY, FL',
       'SAINT LAWRENCE COUNTY, NY', 'SAINT LUCIE COUNTY, FL'],
      dtype=object)

***A Lot of manual digging took place from this moment forward. Its removed. Couple things identified:***

* De Baca County, NM is DeBaca County, NM
* Saint Lawrence County, NY is St. Lawrence County, NY
* De Kalb County, AL is DeKalb County, AL
* De Soto County, FL is DeSoto County, FL
* De Witt County, TX is Dewitt County, TX
* Dade County, FL is Miami-Dade County, FL
* Saint Clair County, AL is St. Clair County, AL
* Saint Johns County, FL is St. Johns County, FL
* Saint Lucie County, FL is St. Lucie County, FL

In [83]:
"DONA ANA COUNTY, NM" in wapo_df_fips["COUNTY_NAME"].unique()

True

In [84]:
"DOÑA ANA COUNTY, NM" in wapo_df_fips["COUNTY_NAME"].unique()

False

In [25]:
# show counties that are in the missing counties list
# wapo_df[wapo_df["COUNTY_NAME"].isin(missing_counties)]


In [26]:
# fix the names
wapo_df["COUNTY_NAME"] = wapo_df["COUNTY_NAME"].replace(
    {
        "DE KALB COUNTY, AL": "DEKALB COUNTY, AL",
        "DE SOTO COUNTY, FL": "DESOTO COUNTY, FL",
        "DE WITT COUNTY, TX": "DEWITT COUNTY, TX",
        "MIAMI-DADE COUNTY, FL": "DADE COUNTY, FL",
        "SAINT CLAIR COUNTY, AL": "ST. CLAIR COUNTY, AL",
        "SAINT JOHNS COUNTY, FL": "ST. JOHNS COUNTY, FL",
        "SAINT LUCIE COUNTY, FL": "ST. LUCIE COUNTY, FL",
        "DE BACA COUNTY, NM": "DEBACA COUNTY, NM",
        "SAINT LAWRENCE COUNTY, NY": "ST. LAWRENCE COUNTY, NY",
    }
)


In [27]:
# show counties that are in the missing counties list
wapo_df[wapo_df["COUNTY_NAME"].isin(missing_counties)]


,COUNTY_NAME,STATE,YEAR,QUANTITY


***Success, let's do the merge over***

In [28]:
# Add FIPS codes to the wapo_df data by merging on county_name
wapo_df_fips = pd.merge(wapo_df, fips_df, on="COUNTY_NAME", how="left").copy()


In [29]:
# Drop STATE_FIPS and STATE_y
wapo_df_fips = wapo_df_fips.drop(["STATE_FIPS", "STATE_y"], axis=1)

# strip the last character from the FIPS column
wapo_df_fips["FIPS"] = wapo_df_fips["FIPS"].str[:-1]

# Add the same merge key
wapo_df_fips["merge_key"] = wapo_df_fips["FIPS"] + wapo_df_fips["YEAR"].astype(str)

# rename the QUANTITY column to DRUG_QUANTITY
wapo_df_fips = wapo_df_fips.rename(
    columns={"QUANTITY": "DRUG_QUANTITY", "STATE_x": "STATE"}
)

# add a flag column to indicate the source of the data
wapo_df_fips["source"] = 1


In [30]:
# show rows with NaN
wapo_df_fips[wapo_df_fips["FIPS"].isna()]


,COUNTY_NAME,STATE,YEAR,DRUG_QUANTITY,FIPS,FIPS_INT,merge_key,source


In [31]:
# wapo_df_fips.sample(10)


In [32]:
print(f"There are {wapo_df_fips.shape[0]} rows in the wapo_df_fips")


There are 8947 rows in the wapo_df_fips


#### Merging

***Make the base DF with records for all counties for all years***

In [33]:
# Make the year list from 2003 to 2015
year_list = [i for i in range(2003, 2016)]

# make a base empty dataframe
base_df = pd.DataFrame()

# Make a loop that populates the dataframe with a row for each county and year
for i in fips_df["FIPS"]:
    for j in year_list:
        # concat the new record to the base_df
        base_df = pd.concat(
            [
                base_df,
                pd.DataFrame(
                    {
                        "county_name": fips_df[fips_df["FIPS"] == i][
                            "COUNTY_NAME"
                        ].values[0],
                        "state": fips_df[fips_df["FIPS"] == i]["STATE"].values[0],
                        "year": j,
                        "fips": i,
                        "merge_key": str(i[:-1]) + str(j),
                    },
                    index=[0],
                ),
            ]
        )

# convert merge key to int
# base_df["merge_key"] = base_df["merge_key"].astype(int)

# reset the index
base_df = base_df.reset_index(drop=True)


In [34]:
# base_df.sample(10)

# show DADE COUNTY, FL
# base_df[base_df["county_name"] == "DADE COUNTY, FL"]


In [35]:
assert len(year_list) * fips_df.shape[0] == base_df.shape[0]


***Left merge to add vital stats***

In [36]:
# Left merge the base_df with the vital_df
base_df_vital = pd.merge(base_df, vital_df, on="merge_key", how="left")


In [37]:
# base_df_vital.head(20)


In [38]:
# count rows where drug is not na
base_df_vital["DRUG"].notna().sum()


3064

In [39]:
# make sure miami-dade is gone
# base_df_vital.tail(15)


***We are losing 13 counties from the vital stats merge. We will need to investigate this.***

***Spoiler: it was maimi-dade county***

In [40]:
# show the na county_name
# base_df_vital.tail(15)

# It was miami dade county, fl


In [41]:
assert base_df_vital["DRUG"].sum() == vital_df["DRUG"].sum()


In [42]:
# base_df_vital.tail(15)

# success, drop the extra rows we acquired


In [43]:
# Drop COUNTY_NAME, STATE, FIPS, YEAR, DRUG
base_df_vital = base_df_vital.drop(
    ["COUNTY_NAME", "STATE", "FIPS", "YEAR", "DRUG"], axis=1
)

# rename DEATHS to drug_deaths
base_df_vital = base_df_vital.rename(columns={"DEATHS": "drug_deaths"})


In [44]:
# base_df_vital.sample(10)


***Now merge the wapo df in***

In [45]:
# Left merge the base_df_vital with the wapo_df_fips
base_df_vital_wapo = pd.merge(base_df_vital, wapo_df_fips, on="merge_key", how="outer")


In [46]:
# Looks like we caught all the issues...

# base_df_vital_wapo.tail(10)


In [47]:
# Drop the COUNTY_NAME, STATE, YEAR, FIPS, and FIPS_INT columns
base_df_vital_wapo = base_df_vital_wapo.drop(
    ["COUNTY_NAME", "STATE", "YEAR", "FIPS", "FIPS_INT"], axis=1
)


In [48]:
# all the rows found a home
assert base_df_vital_wapo["source"].sum() == wapo_df_fips["source"].sum()


In [49]:
# drop the source column
base_df_vital_wapo = base_df_vital_wapo.drop("source", axis=1)

# rename the DRUG_QUANTITY column to drug_quantity
base_df_vital_wapo = base_df_vital_wapo.rename(
    columns={"DRUG_QUANTITY": "drug_quantity"}
)


In [50]:
# base_df_vital_wapo.sample(10)


#### Add population

In [51]:
# merge in the population data
base_df_vital_wapo_pop = pd.merge(
    base_df_vital_wapo, census_df, on="county_name", how="left"
)


In [52]:
# show the DADE COUNTY, FL values to verify pop is correct
# base_df_vital_wapo_pop[base_df_vital_wapo_pop["county_name"] == "DADE COUNTY, FL"]


In [53]:
# check tail to make sure miami-dade doesn't creep in
# base_df_vital_wapo_pop.tail(15)


In [54]:
# count the rows where pop is na
# base_df_vital_wapo_pop["POPULATION"].isna().sum()

# drop em
base_df_vital_wapo_pop = base_df_vital_wapo_pop.dropna(subset=["POPULATION"])


In [55]:
assert base_df_vital_wapo_pop.POPULATION.isna().sum() == 0


In [56]:
# show na population rows
base_df_vital_wapo_pop[base_df_vital_wapo_pop["POPULATION"].isna()]


,county_name,state,year,fips,merge_key,drug_deaths,drug_quantity,POPULATION,STATE


In [57]:
# drop the STATE column
base_df_vital_wapo_pop = base_df_vital_wapo_pop.drop("STATE", axis=1)

# rename the POPULATION column to population
base_df_vital_wapo_pop = base_df_vital_wapo_pop.rename(
    columns={"POPULATION": "population"}
)


In [58]:
# base_df_vital_wapo_pop.sample(10)


#### Add columns for per capita rates

In [59]:
grouped_df = base_df_vital_wapo_pop.copy()


In [60]:
# add a column for death rate per 100,000 people
grouped_df["death_rate"] = grouped_df["drug_deaths"] / grouped_df["population"] * 100000

# add a column for ship rate per 100,000 people
grouped_df["ship_rate"] = (
    grouped_df["drug_quantity"] / grouped_df["population"] * 100000
)


In [86]:
grouped_df.sample(10)


,county_name,state,year,fips,merge_key,drug_deaths,drug_quantity,population,death_rate,ship_rate
573,"MADISON COUNTY, AL",AL,2004,01089,010892004,12.0,NaN,334811.0,3.584112,NaN
10659,"FANNIN COUNTY, TX",TX,2015,48147,481472015,NaN,NaN,33915.0,NaN,NaN
4386,"TALIAFERRO COUNTY, GA",GA,2008,13265,132652008,NaN,NaN,1717.0,NaN,NaN
12566,"TAYLOR COUNTY, TX",TX,2011,48441,484412011,14.0,35632.0,131506.0,10.645902,27095.341657
10188,"CHILDRESS COUNTY, TX",TX,2012,48075,480752012,NaN,3025.0,7041.0,NaN,42962.647351
8288,"MUSKOGEE COUNTY, OK",OK,2010,40101,401012010,16.0,21965.0,70990.0,22.538386,30940.977602
3115,"COBB COUNTY, GA",GA,2011,13067,130672011,65.0,193491.0,688078.0,9.446603,28120.503780
10300,"COMANCHE COUNTY, TX",TX,2007,48093,480932007,NaN,2296.0,13974.0,NaN,16430.513811
12951,"YOAKUM COUNTY, TX",TX,2006,48501,485012006,NaN,856.0,7879.0,NaN,10864.322884
8200,"MCCLAIN COUNTY, OK",OK,2013,40087,400872013,NaN,15041.0,34506.0,NaN,43589.520663


***Add assert tests***

In [90]:
# assert len(grouped_df["county_name"]) == 1039
# FALSE
len(grouped_df["county_name"].unique())

1037

In [91]:
"DONA ANA COUNTY, NM" in grouped_df["county_name"]

False

In [92]:
"DOÑA ANA COUNTY, NM" in grouped_df["county_name"]

False

In [72]:
missing_counties =  set(fips_df["COUNTY_NAME"].unique()) - set(grouped_df["county_name"].unique())
print(f"We missed entire data points for {missing_counties}")

We missed entire data points for {'DEBACA COUNTY, NM', 'DONA ANA COUNTY, NM'}


#### Export the final grouped df

In [62]:
# export to the 20_intermediate_files directory
grouped_df.to_csv("../20_intermediate_files/analysis_df.csv", index=False)
